In [ ]:
import matplotlib.pyplot as plt
from hypertiling import HyperbolicTiling
from hypertiling.graphics.svg import *
import hypertiling.graphics.plot as htplot
from hypertiling.geodesics import geodesic_midpoint as midpt
import numpy as np
import itertools
import math
from IPython.display import clear_output

In [ ]:

def endPointsGeodesic(z1,z2):
    a = (np.imag(z1)*(np.abs(z2)**2+1)-np.imag(z2)*(np.abs(z1)**2+1))/(np.real(z1)*np.imag(z2)-np.real(z2)*np.imag(z1))
    b = (np.real(z2)*(np.abs(z1)**2+1)-np.real(z1)*(np.abs(z2)**2+1))/(np.real(z1)*np.imag(z2)-np.real(z2)*np.imag(z1))
    if a==0:
        y0 = -2/b
        x01 = np.sqrt(1-4/b**2)
        x02 = -np.sqrt(1-4/b**2)
        return (x01+1j*y0,x02+y0*1j)
    elif b == 0:
        x0 = -2/a
        y01 = np.sqrt(1-4/a**2)
        y02 = -np.sqrt(1-4/a**2)
        return (x0+1j*y01,x0+1j*y02)
    else:
        x0 = -(np.sqrt(b*b*(a*a+b*b-4))+2*a)/(a*a+b*b)
        y0 = (a*np.sqrt(b*b*(a*a+b*b-4))-2*b*b)/(b*(a*a+b*b))
        x01 = (np.sqrt(b*b*(a*a+b*b-4))-2*a)/(a*a+b*b)
        y01 = -(a*np.sqrt(b*b*(a*a+b*b-4))+2*b*b)/(b*(a*a+b*b))
        return np.array([x0+y0*1j,x01+y01*1j])

def geodesic_distance_to_origin(z1,z2):
    a = (np.imag(z1)*(np.abs(z2)**2+1)-np.imag(z2)*(np.abs(z1)**2+1))/(np.real(z1)*np.imag(z2)-np.real(z2)*np.imag(z1))
    b = (np.real(z2)*(np.abs(z1)**2+1)-np.real(z1)*(np.abs(z2)**2+1))/(np.real(z1)*np.imag(z2)-np.real(z2)*np.imag(z1))
    if b==0:
        return min(np.abs(np.sqrt(-1+a*a/4)-a/2),np.abs(-np.sqrt(-1+a*a/4)-a/2))
    if a==0:
        return min(np.abs(np.sqrt(-1+b*b/4)-b/2),np.abs(-np.sqrt(-1+b*b/4)-b/2))
    else:
        u = np.sqrt(np.power(a,6)+2*np.power(a,4)*(b*b-2)+a*a*b*b*(b*b-4))
        x0 = -(np.power(a,3)+u+a*b*b)/(2*(a*a+b*b))
        y0 = -b*(np.power(a,3)+u+a*b*b)/(2*a*(a*a+b*b))
        x1 = -(np.power(a,3)-u+a*b*b)/(2*(a*a+b*b))
        y1 = -b*(np.power(a,3)-u+a*b*b)/(2*a*(a*a+b*b))
        return min(np.sqrt(x0*x0+y0*y0),np.sqrt(x1*x1+y1*y1))

    
def radial_distance_polar(z):
    return np.abs(z)

def establish_boundary(tiling, cutoff_radius):
    boundary = np.zeros(len(T)).astype("bool")
    for j in range(len(T)):
        if radial_distance_polar(T.get_center(j)) > cutoff_radius:
            boundary[j] = True
    return boundary


def my_angular_filter(z, boundary = 1, inner = 0.1,deg_min = 0, deg_max = 90):
    return True if (deg_min < np.angle(z, deg=True) < deg_max and radial_distance_polar(z) < boundary and radial_distance_polar(z)>inner) else False

def quick_plot(tiling, unitcircle=False, fig = None, ax = None,dpi=150,colors = None):
    if colors == None:
        colors = ['w' for _ in range(len(tiling))]
    if len(colors) == 1:
        fill_color = colors[0]
        colors = [fill_color for _ in range(len(tiling))]
    # actual plot
    if fig == None and ax==None:
        fig, ax = plt.subplots(figsize=(8, 7), dpi=dpi)
        plt.xlim([-1, 1])
        plt.ylim([-1, 1])
        plt.axis('equal')
        plt.axis('off')
    
    # add bounding circle
    if unitcircle:
        circle = plt.Circle((0, 0), 1)
        ax.add_patch(circle)

    # draw polygons
    x, y = [], []
    for i in range(len(tiling)):
        v = tiling.get_vertices(i)
        v = np.append(v, v[0])  # appending first vertex to close the path
        x.extend(v.real)
        x.append(None)  # this is some kind of trick that makes it that fast
        y.extend(v.imag)
        y.append(None)
    polygons = [list(group) for key, group in itertools.groupby(zip(x, y), lambda x: x[0] is None) if not key]

    for i, polygon in enumerate(polygons):
        x0, y0 = zip(*polygon)
        ax.fill(x0, y0, color=colors[i],edgecolor ='k')
    return (fig,ax)

def flip(c):
    if c == 'r':
        return 'b'
    if c=='b':
        return 'r'
    
def cross(z1,z2):
        return z1.real*z2.imag-z2.real*z1.imag
def is_between(z0,z1,z2):
    return (cross(z1,z0)*cross(z1,z2) >= 0 and cross(z2,z0)*cross(z2,z1) >= 0)


In [ ]:

p = 5; q = 4; nlayers = 5
tiling = HyperbolicTiling(p,q,nlayers)
#fig, ax = quick_plot(tiling,unitcircle= True,colors = 'r')

import time

colors = ['r' for _ in range(len(tiling))]
outers = [index for index in range(len(tiling)) if len(tiling.get_nbrs(index))!=5 ]
bulk = [index for index in range(len(tiling)) if len(tiling.get_nbrs(index)) ==5]
geodesicas = []
for i,index in enumerate(bulk):
    for vertex in range(5):
        vertices = (tiling.get_vertices(index)[vertex],tiling.get_vertices(index)[(vertex+1)%5])
        geodesicends = endPointsGeodesic(vertices[0],vertices[1])
        minrad = geodesic_distance_to_origin(vertices[0],vertices[1])
        g_new = (geodesicends,minrad)
        add = True
        for g in geodesicas:
            if np.abs(g[0][0]-g_new[0][0]) < 0.01 and np.abs(g[0][1]-g_new[0][1]) <0.01 :
                add = False
            if np.abs(g[0][1]-g_new[0][0]) < 0.01 and np.abs(g[0][0]-g_new[0][1]) <0.01 :
                add = False
        if add:
            geodesicas.append(g_new)    

for iteration in range(1):
    pol = np.random.choice(bulk)
    side = np.random.randint(5)
    geodesic = (tiling.get_vertices(pol)[side],tiling.get_vertices(pol)[(side+1)%5])
    geodesicends = endPointsGeodesic(geodesic[1],geodesic[0])
    midpoint = midpt(geodesic[0],geodesic[1])
    minrad = geodesic_distance_to_origin(geodesic[0],geodesic[1])
    inner = np.random.randint(2)
    for index in range(len(tiling)):
        center = tiling.get_center(index)

        if inner==1:
            if(np.abs(center)>minrad and is_between(center,geodesicends[0],geodesicends[1])):
                colors[index] = flip(colors[index])
        if inner==0:
            if(np.abs(center)>minrad and is_between(center,geodesicends[0],geodesicends[1])):
                continue
            else:
                colors[index] = flip(colors[index])
    #clear_output(wait=True)
    fig, ax = quick_plot(tiling,unitcircle = True,colors = colors)
    #ax.plot([0,geodesicends[0].real],[0,geodesicends[0].imag],zorder=4)
    #ax.plot([0,geodesicends[1].real],[0,geodesicends[1].imag],zorder=4)
    time.sleep(1)
    
